In [15]:
!pip install -r local.requirements.txt 2>/dev/null

In [21]:
import pandas as pd
import numpy as np

from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitItemKNNWrapperModel

from implicit.nearest_neighbours import TFIDFRecommender

## Скачиваем архив с датасетом и сохраняем в DataFrame

In [17]:
%%time
!wget -q https://files.grouplens.org/datasets/movielens/ml-1m.zip -O ml-1m.zip
!unzip -o ml-1m.zip
!rm ml-1m.zip

Archive:  ml-1m.zip
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
CPU times: user 88.5 ms, sys: 31.2 ms, total: 120 ms
Wall time: 5.57 s


In [18]:
movies = pd.read_csv(
    "ml-1m/movies.dat", 
    sep="::",
    engine="python",
    header=None,
    names=[Columns.Item, "title", "genres"],
    encoding_errors="ignore",
)

ratings = pd.read_csv(
    "ml-1m/ratings.dat", 
    sep="::",
    engine="python",
    header=None,
    names=[Columns.User, Columns.Item, Columns.Weight, Columns.Datetime],
)



In [19]:
movies.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
ratings.head()

,user_id,item_id,weight,datetime
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Собираем модель

In [25]:
ratings_dataset = Dataset.construct(ratings)

model = ImplicitItemKNNWrapperModel(TFIDFRecommender(K=10))
model.fit(ratings_dataset)
basic_recos = model.recommend(
    users=ratings[Columns.User].unique(),
    dataset=ratings_dataset,
    k=10,
    filter_viewed=True,
)

basic_recos.head()

,user_id,item_id,score,rank
0,1,364,20.436578,1
1,1,1196,15.716834,2
2,1,318,15.625371,3
3,1,2096,14.876911,4
4,1,2571,12.718620,5


In [29]:
# Select random user, see history of views and reco for this user
USER_ID = 2023

query_recos = basic_recos.query("user_id == @USER_ID").merge(movies, on="item_id")
query_viewed = ratings.query("user_id == @USER_ID").merge(movies, on="item_id")

query_recos.sort_values("rank")

,user_id,item_id,score,rank,title,genres
0,2023,2916,40.452390,1,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
1,2023,1200,28.331182,2,Aliens (1986),Action|Sci-Fi|Thriller|War
2,2023,2396,27.476134,3,Shakespeare in Love (1998),Comedy|Romance
3,2023,1214,25.924201,4,Alien (1979),Action|Horror|Sci-Fi|Thriller
4,2023,733,22.115749,5,"Rock, The (1996)",Action|Adventure|Thriller
5,2023,1079,21.636740,6,"Fish Called Wanda, A (1988)",Comedy
6,2023,2683,21.632333,7,Austin Powers: The Spy Who Shagged Me (1999),Comedy
7,2023,1394,18.386288,8,Raising Arizona (1987),Comedy
8,2023,1307,17.446031,9,When Harry Met Sally... (1989),Comedy|Romance
9,2023,2355,13.991752,10,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [30]:
query_viewed.query("weight > 3")

,user_id,item_id,weight,datetime,title,genres
0,2023,2987,4,1006290527,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
3,2023,720,5,974672276,Wallace & Gromit: The Best of Aardman Animatio...,Animation
4,2023,1257,4,974673156,Better Off Dead... (1985),Comedy
5,2023,2997,5,974673112,Being John Malkovich (1999),Comedy
7,2023,1259,5,1006290472,Stand by Me (1986),Adventure|Comedy|Drama
...,...,...,...,...,...,...
278,2023,1095,5,1015127722,Glengarry Glen Ross (1992),Drama
279,2023,1097,4,1006290808,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
280,2023,709,4,1006290704,Oliver & Company (1988),Animation|Children's
281,2023,569,4,1006290864,Little Big League (1994),Children's|Comedy


In [31]:
query_recos

,user_id,item_id,score,rank,title,genres
0,2023,2916,40.452390,1,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
1,2023,1200,28.331182,2,Aliens (1986),Action|Sci-Fi|Thriller|War
2,2023,2396,27.476134,3,Shakespeare in Love (1998),Comedy|Romance
3,2023,1214,25.924201,4,Alien (1979),Action|Horror|Sci-Fi|Thriller
4,2023,733,22.115749,5,"Rock, The (1996)",Action|Adventure|Thriller
5,2023,1079,21.636740,6,"Fish Called Wanda, A (1988)",Comedy
6,2023,2683,21.632333,7,Austin Powers: The Spy Who Shagged Me (1999),Comedy
7,2023,1394,18.386288,8,Raising Arizona (1987),Comedy
8,2023,1307,17.446031,9,When Harry Met Sally... (1989),Comedy|Romance
9,2023,2355,13.991752,10,"Bug's Life, A (1998)",Animation|Children's|Comedy
